In [1]:
from datetime import date, timedelta
from itertools import product
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import os,sys
os.chdir('/data/arpit.goel/33_GrocerySales')

log_sales=lambda u: np.log1p(float(u)) if float(u) > 0 else 0
dtypes={'onpromotion': bool}

train = pd.read_csv('01.RawData/train.csv', usecols=range(1,6),dtype=dtypes, 
    converters={'unit_sales': log_sales},parse_dates=['date'],skiprows=range(1,101688780),
    index_col=['store_nbr', 'item_nbr', 'date'])

test = pd.read_csv('01.RawData/test.csv', usecols=range(5),dtype=dtypes,
    parse_dates=['date'],index_col=['store_nbr', 'item_nbr', 'date'])

items = pd.read_csv('01.RawData/items.csv',index_col=['item_nbr'])
promo_2017=pd.concat([train['onpromotion'],test['onpromotion']]).unstack().fillna(False)
sales_2017=train['unit_sales'].unstack().fillna(0)
promosales_2017=(train['unit_sales']*train['onpromotion']).unstack().fillna(0)
promosales_2017=promosales_2017.replace(-999, np.nan)
promo_2017=promo_2017.reindex(sales_2017.index).fillna(False)
min_dates=train.reset_index().groupby(['store_nbr', 'item_nbr'])['date'].min()
min_dates.name='first_date'
index=promo_2017.reset_index()[['store_nbr','item_nbr']]
num_promo=promo_2017.stack().unstack(1).sum(axis=1).unstack()
num_promo=(num_promo.T/index['store_nbr'].value_counts().sort_index()).T
num_promo=pd.merge(index[['store_nbr']],num_promo,left_on='store_nbr',right_index=True)
num_promo=num_promo[num_promo.columns[1:]]
num_promo.columns=promo_2017.columns


In [2]:
max_sales=sales_2017.max(axis=1)
max_sales_count=(sales_2017.T==max_sales).sum().T
sales_2017['max_sales']=max_sales
sales_2017['max_sales_count']=max_sales_count
max_sales=sales_2017[sales_2017['max_sales_count']>1]
max_sales=np.expm1(max_sales)
max_sales['max_sales_count']= np.log1p(max_sales['max_sales_count'])
max_sales['sales_count']=max_sales[sales_2017.columns].apply(lambda x: np.unique(x).shape[0],axis=1)
max_sales=max_sales[max_sales['sales_count']>4]

In [17]:
max_sales.sort_values(by=['max_sales_count'],ascending=False).to_csv('03.Profile/03.MaxSales.csv')

In [40]:
train = pd.read_csv('01.RawData/train.csv', usecols=range(1,6),dtype=dtypes
    ,parse_dates=['date'],skiprows=range(1,101688780),
    index_col=['store_nbr', 'item_nbr'])
train=train[train.date.dt.month==8]
train=train[train.date.dt.day>=9]

In [67]:
train_1=train.copy()
train_1['date']=train_1['date'].map(lambda x: x+timedelta(days=7))
train_2=train.copy()
train_2['date']=train_2['date'].map(lambda x: x+timedelta(days=14))
train_3=train.copy()
train_3['date']=train_3['date'].map(lambda x: x+timedelta(days=21))
prediction=pd.concat([train_1,train_2,train_3]).reset_index()

In [68]:
prediction.date.value_counts().sort_index()

2017-08-16    100992
2017-08-17     98552
2017-08-18    104599
2017-08-19    105192
2017-08-20    106104
2017-08-21    102750
2017-08-22    102812
2017-08-23    100992
2017-08-24     98552
2017-08-25    104599
2017-08-26    105192
2017-08-27    106104
2017-08-28    102750
2017-08-29    102812
2017-08-30    100992
2017-08-31     98552
2017-09-01    104599
2017-09-02    105192
2017-09-03    106104
2017-09-04    102750
2017-09-05    102812
Name: date, dtype: int64

In [69]:
test=pd.read_csv('01.RawData/test.csv',dtype=dtypes
    ,parse_dates=['date'])
test=pd.merge(test,prediction,on=['store_nbr','item_nbr','date'],how='left')

In [70]:
test['unit_sales']=np.clip(test['unit_sales'].fillna(0),0,10000)
test[['id','unit_sales']].to_csv('04.Submissions/09.PreviousWeekSales.csv.gz',compression='gzip',index=False)

In [71]:
test[['id','unit_sales']].shape

(3370464, 2)

In [72]:
test[['id','unit_sales']].drop_duplicates().shape

(3370464, 2)

In [57]:
! zcat 04.Submissions/09.PreviousWeekSales.csv.gz | head

id,unit_sales
125497040,0.0
125497041,2.0
125497042,0.0
125497043,0.0
125497044,7.0
125497045,6.0
125497046,11.0
125497047,0.0
125497048,1.0

gzip: stdout: Broken pipe


In [32]:
is_numerical=(sales_2017==sales_2017.astype(np.int64)).product(axis=1)
is_numerical=is_numerical.reset_index()

In [39]:
a=is_numerical.groupby(['item_nbr'])[0].agg([np.max,np.min])
pd.crosstab(a.iloc[:,0],a.iloc[:,1])

amin,0,1
amax,,
0,277,0
1,16,3725
